[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HesusG/diagnostico-lineas-accion/blob/main/Semana2/ejercicios/workshop2_plantilla.ipynb)

# Workshop 2: Pruebas Estadísticas Avanzadas

**Curso:** CD2001B  
**Módulo:** 2 - Pruebas de Hipótesis

## 🎯 Rúbrica

| Parte | Puntos |
|-------|--------|
| Chi-Cuadrada | 30 |
| ANOVA | 35 |
| Regresión | 35 |
| **TOTAL** | **100** |

> **Nota:** Algunas visualizaciones ya están pre-completadas para ahorrar tiempo. Enfócate en la ejecución de las pruebas y su interpretación.

## 0. Información del Estudiante

**Nombre:** [Tu nombre]  
**Matrícula:** [Tu matrícula]

## 1. Preparación

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
sns.set_palette('Set2')
%matplotlib inline

print('✓ Librerías OK')

In [ ]:
# Cargar dataset
df = pd.read_csv('https://raw.githubusercontent.com/HesusG/diagnostico-lineas-accion/main/Semana1/datos/ejemplo_satisfaccion_clientes.csv')

print(f'Dataset: {df.shape[0]} beneficiarios')
df.head()

---

## PARTE 1: Prueba de Chi-Cuadrada (χ²) - 30 puntos

### 📚 Contexto Teórico

La **prueba de Chi-Cuadrada** determina si existe una **relación significativa** entre dos variables categóricas.

**¿Cuándo usarla?**
- Ambas variables son **categóricas** (género, área, satisfacción_categoria)
- Queremos saber si una variable **depende** de la otra

**Hipótesis:**
- **H₀:** Las variables son **independientes** (no hay relación)
- **H₁:** Las variables son **dependientes** (sí hay relación)

<details>
<summary>💡 <b>Hint: Interpretación del p-value</b></summary>

<br>

- **p-value < 0.05**: Rechazamos H₀ → Las variables SÍ están relacionadas
- **p-value ≥ 0.05**: No rechazamos H₀ → No hay evidencia de relación

**Ejemplo práctico:**
Si p-value = 0.003 para "género vs área", significa que ciertos géneros prefieren ciertas áreas (la relación NO es por azar).

</details>

---

### Pregunta de Investigación

**¿Existe relación entre el GÉNERO y el ÁREA de servicio solicitada?**

---

### 1.1 Tabla de Contingencia (10 puntos)

Crea una tabla de contingencia que muestre la frecuencia de beneficiarios por género y área.

In [ ]:
# TU CÓDIGO AQUÍ
# Crear tabla de contingencia entre 'genero' y 'area'



**Pregunta 1.1:** Observa la tabla. ¿Parece haber algún patrón? ¿Algún área es claramente preferida por un género?

**Tu respuesta:**  
[Escribe tu observación aquí]

---

### 1.2 Visualización con Heatmap (PRE-COMPLETADO ✓)

Este heatmap te ayudará a visualizar los patrones en la tabla de contingencia.

> **Nota:** Ya está completado para ahorrar tiempo. Observa los patrones para tu interpretación posterior.

In [ ]:
# PRE-COMPLETADO: Heatmap de tabla de contingencia

plt.figure(figsize=(10, 6))

sns.heatmap(tabla_contingencia, annot=True, fmt='d', cmap='YlOrRd', 
            linewidths=0.5, cbar_kws={'label': 'Frecuencia'})

plt.title('Distribución de Género por Área de Servicio', 
          fontsize=14, fontweight='bold')
plt.ylabel('Género', fontsize=12)
plt.xlabel('Área de Servicio', fontsize=12)
plt.tight_layout()
plt.show()

### 1.3 Prueba Chi-Cuadrada (15 puntos)

Ejecuta la prueba para determinar si hay relación estadísticamente significativa.

In [ ]:
# TU CÓDIGO AQUÍ
# Realizar prueba de Chi-Cuadrada



### 1.4 Interpretación MEAL (5 puntos)

**Pregunta 1.4:** Interpreta los resultados usando el framework MEAL:

**M (Main Idea):**  
[¿Hay o no relación entre género y área?]

**E (Evidence):**  
[Cita el χ² y p-value]

**A (Analysis):**  
[¿Qué patrones observaste en el heatmap? ¿Qué áreas prefiere cada género?]

**L (Link):**  
[¿Cómo puede la ONG usar esta información? ¿Qué acciones tomar?]

---

---

## PARTE 2: ANOVA (Análisis de Varianza) - 35 puntos

### 📚 Contexto Teórico

**ANOVA** nos permite comparar las medias de **3 o más grupos** simultáneamente.

**¿Cuándo usarla?**
- Variable independiente: **categórica** con 3+ grupos (ej: área con Legal, Trámites, Psicología)
- Variable dependiente: **numérica** (ej: satisfacción, edad)

**Hipótesis:**
- **H₀:** Todas las medias son iguales (μ₁ = μ₂ = μ₃ = ...)
- **H₁:** Al menos una media es diferente

<details>
<summary>💡 <b>Hint: ANOVA vs prueba t</b></summary>

<br>

**Prueba t:** Compara 2 grupos  
**ANOVA:** Compara 3+ grupos simultáneamente

**¿Por qué no hacer múltiples pruebas t?**  
Hacer muchas pruebas t aumenta el **error Tipo I** (falsos positivos). ANOVA controla este error.

**Si ANOVA es significativa:** Hacemos pruebas post-hoc (ej: Tukey) para saber **qué pares** difieren.

</details>

---

### Pregunta de Investigación

**¿Existen diferencias significativas en la SATISFACCIÓN promedio entre las diferentes ÁREAS de servicio?**

---

### 2.1 Estadísticas Descriptivas por Grupo (10 puntos)

Calcula la media y desviación estándar de satisfacción para cada área.

In [ ]:
# TU CÓDIGO AQUÍ
# Calcular estadísticas por área



**Pregunta 2.1:** Observa las medias. ¿Parecen ser diferentes? ¿Cuál área tiene mayor satisfacción?

**Tu respuesta:**  
[Tu observación aquí]

---

### 2.2 Visualización Comparativa (PRE-COMPLETADO ✓)

Este boxplot te permite comparar visualmente las distribuciones de satisfacción por área.

> **Nota:** Ya está completado. Observa qué áreas están por encima/debajo de la media general.

In [ ]:
# PRE-COMPLETADO: Boxplot comparativo

plt.figure(figsize=(12, 6))

sns.boxplot(data=df, x='area', y='satisfaccion', palette='Set2')

# Agregar línea de media general
media_general = df['satisfaccion'].mean()
plt.axhline(y=media_general, color='red', linestyle='--', 
            linewidth=2, label=f'Media General = {media_general:.2f}')

plt.title('Distribución de Satisfacción por Área', fontsize=14, fontweight='bold')
plt.xlabel('Área de Servicio', fontsize=12)
plt.ylabel('Satisfacción (1-10)', fontsize=12)
plt.ylim(0, 10)
plt.legend()
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

### 2.3 Ejecución de ANOVA (12 puntos)

Realiza la prueba ANOVA para determinar si las diferencias son estadísticamente significativas.

In [ ]:
# TU CÓDIGO AQUÍ
# Preparar grupos y realizar ANOVA



### 2.4 Comparaciones Pareadas (Post-hoc) (8 puntos)

Si ANOVA fue significativa, realiza comparaciones por pares con pruebas t.

In [ ]:
# TU CÓDIGO AQUÍ
# Comparaciones por pares (si ANOVA fue significativa)



### 2.5 Interpretación MEAL (5 puntos)

**Pregunta 2.5:** Interpreta los resultados completos:

**M (Main Idea):**  
[¿Existen diferencias significativas entre las áreas?]

**E (Evidence):**  
[Cita el estadístico F, p-value, y diferencias de medias]

**A (Analysis):**  
[Si hay diferencias, ¿entre qué áreas específicamente? ¿Cuál área destaca?]

**L (Link):**  
[¿Qué debe hacer la ONG? ¿Investigar por qué una área tiene baja satisfacción? ¿Replicar buenas prácticas?]

---

---

## PARTE 3: Regresión Lineal - 35 puntos

### 📚 Contexto Teórico

La **Regresión Lineal** modela la relación entre una variable dependiente (Y) y una o más independientes (X).

**¿Cuándo usarla?**
- Variable dependiente: **numérica continua** (ej: satisfacción)
- Variables independientes: **numéricas** (ej: edad, tiempo_espera)
- Queremos **predecir** Y basándonos en X

**Ecuación:** Y = β₀ + β₁X₁ + β₂X₂ + ... + ε

**Métricas clave:**
- **R² (coeficiente de determinación):** % de variabilidad explicada (0 a 1)
  - R² = 0.7 → El modelo explica 70% de la variabilidad
- **Coeficientes (β):** Cambio en Y por cada unidad de cambio en X

<details>
<summary>💡 <b>Hint: Interpretación de coeficientes</b></summary>

<br>

**Ejemplo:** Satisfacción = 5.0 + 0.3×edad - 0.5×tiempo_espera

**Interpretación:**
- **Intercepto (5.0):** Satisfacción base cuando edad=0 y tiempo_espera=0
- **β₁ (0.3):** Por cada año adicional de edad, satisfacción aumenta 0.3 puntos
- **β₂ (-0.5):** Por cada minuto adicional de espera, satisfacción disminuye 0.5 puntos

**Coeficiente negativo** = relación inversa (a más X, menos Y)  
**Coeficiente positivo** = relación directa (a más X, más Y)

</details>

---

### Pregunta de Investigación

**¿Podemos predecir la SATISFACCIÓN basándonos en EDAD, CALIDAD_ATENCIÓN y TIEMPO_ESPERA?**

---

### 3.1 Análisis de Correlación Inicial (7 puntos)

Antes de crear el modelo, calcula las correlaciones entre variables.

> **Nota:** El cálculo es tu tarea; la visualización con heatmap está pre-completada.

In [ ]:
# TU CÓDIGO AQUÍ
# Calcular matriz de correlación

vars_interes = ['satisfaccion', 'edad', 'calidad_atencion', 'tiempo_espera']
# Completa el cálculo de correlaciones



In [ ]:
# PRE-COMPLETADO: Visualización del heatmap de correlación

plt.figure(figsize=(8, 6))
sns.heatmap(correlaciones, annot=True, cmap='coolwarm', 
            center=0, vmin=-1, vmax=1, square=True, 
            linewidths=1, cbar_kws={'label': 'Correlación'})

plt.title('Matriz de Correlación', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

**Pregunta 3.1:** ¿Qué variables tienen mayor correlación con satisfacción? ¿Son positivas o negativas?

**Tu respuesta:**  
[Tu análisis aquí]

---

### 3.2 Preparación de Datos (5 puntos)

Prepara las variables X (predictoras) y y (objetivo).

In [ ]:
# TU CÓDIGO AQUÍ
# Preparar X e y para el modelo



### 3.3 Creación del Modelo (10 puntos)

Crea y entrena un modelo de regresión lineal.

In [ ]:
# TU CÓDIGO AQUÍ
# Crear y entrenar modelo

from sklearn.linear_model import LinearRegression



### 3.4 Evaluación del Modelo (13 puntos)

Calcula R², RMSE y analiza los coeficientes.

In [ ]:
# TU CÓDIGO AQUÍ
# Evaluar modelo (R² y RMSE)

from sklearn.metrics import r2_score, mean_squared_error



In [ ]:
# TU CÓDIGO AQUÍ
# Analizar coeficientes del modelo



### 3.5 Visualización del Modelo (PRE-COMPLETADO ✓)

Este scatter plot compara valores reales vs predichos para evaluar visualmente el modelo.

> **Nota:** Ya está completado. Observa qué tan cerca están los puntos de la línea roja.

In [ ]:
# PRE-COMPLETADO: Gráfico Real vs Predicho

plt.figure(figsize=(10, 6))

# Scatter plot con seaborn
sns.scatterplot(x=y, y=y_pred, alpha=0.6, s=80, color='steelblue')

# Línea de predicción perfecta (y=x)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 
         'r--', lw=2, label='Predicción Perfecta')

plt.title(f'Valores Reales vs Predichos (R² = {r2:.3f})', 
          fontsize=14, fontweight='bold')
plt.xlabel('Satisfacción Real', fontsize=12)
plt.ylabel('Satisfacción Predicha', fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print('\n💡 Interpretación:')
print('• Puntos cerca de la línea roja = Predicciones precisas')
print('• Puntos alejados = El modelo no predice bien esos casos')

### 3.6 Interpretación MEAL Completa (5 puntos → Renumerado como 3.5)

**Pregunta 3.5:** Interpreta el modelo completo:

**M (Main Idea):**  
[¿El modelo es útil para predecir satisfacción? ¿R² es aceptable?]

**E (Evidence):**  
[Cita R², RMSE, y los coeficientes más importantes]

**A (Analysis):**  
[¿Qué variable tiene más impacto? ¿Las relaciones son las esperadas? ¿Qué sorprende?]

**L (Link):**  
[¿Cómo usar este modelo? ¿Qué debe priorizar la ONG para mejorar satisfacción?]

---

---

## PARTE 4: Reflexión Final MEAL (BONUS +10 puntos)

### Síntesis Integral del Workshop

Imagina que eres el **analista de datos principal** de la ONG. El director ejecutivo te pide un **reporte ejecutivo** integrando los tres análisis realizados.

**Pregunta BONUS:** Escribe un reporte ejecutivo estructurado con MEAL que integre:
- Los hallazgos de Chi-Cuadrada (relaciones entre variables categóricas)
- Los hallazgos de ANOVA (diferencias entre áreas)
- Los hallazgos de Regresión (factores que predicen satisfacción)

**Estructura tu respuesta:**

**M (Main Idea - Mensaje Central):**  
[Resume en 2-3 oraciones el hallazgo más importante del workshop completo]

**E (Evidence - Evidencia Integrada):**  
[Cita los 3 p-values/R² más relevantes de cada prueba]

**A (Analysis - Análisis Cruzado):**  
[¿Cómo se relacionan los 3 análisis? ¿Qué historia cuentan juntos? ¿Hay patrones consistentes?]

**L (Link - Recomendaciones Accionables):**  
[Lista 3 acciones concretas prioritarias para la ONG basadas en los datos]

**Tu respuesta (mínimo 200 palabras):**  
[Escribe tu reporte ejecutivo aquí]

---

---

## ✅ Checklist Antes de Entregar

Verifica que completaste todo:

### Parte 1: Chi-Cuadrada
- [ ] Tabla de contingencia creada correctamente
- [ ] Prueba χ² ejecutada con resultados
- [ ] Interpretación MEAL completa

### Parte 2: ANOVA
- [ ] Estadísticas descriptivas por grupo calculadas
- [ ] Prueba ANOVA ejecutada
- [ ] Comparaciones post-hoc realizadas (si aplica)
- [ ] Interpretación MEAL completa

### Parte 3: Regresión
- [ ] Matriz de correlación calculada
- [ ] Datos preparados (X e y)
- [ ] Modelo entrenado correctamente
- [ ] R², RMSE y coeficientes calculados
- [ ] Interpretación MEAL completa

### General
- [ ] Todas las celdas ejecutadas con resultados visibles
- [ ] Sin errores de ejecución
- [ ] Interpretaciones en español claro
- [ ] Respondiste TODAS las preguntas
- [ ] Reflexión BONUS completada (opcional)
- [ ] Revisaste ortografía

---

---

## 📤 Formato de Entrega

**Nombre de archivo:**  
`Workshop2_NombreApellido_Matricula.ipynb`

**Ejemplo:**  
`Workshop2_JuanPerez_A01234567.ipynb`

**Fecha límite:** [Consulta con tu profesor]

---

## 🎯 Criterios de Evaluación Detallados

### Chi-Cuadrada (30 pts)
- Tabla de contingencia correcta (10 pts)
- Prueba ejecutada correctamente (15 pts)
- Interpretación contextual (5 pts)

### ANOVA (35 pts)
- Estadísticas descriptivas (10 pts)
- Prueba ANOVA correcta (12 pts)
- Post-hoc apropiado (8 pts)
- Interpretación contextual (5 pts)

### Regresión (35 pts)
- Análisis de correlación (7 pts)
- Preparación de datos (5 pts)
- Modelo creado y entrenado (10 pts)
- Evaluación completa (R², RMSE, coeficientes) (13 pts)

### BONUS (10 pts)
- Reflexión MEAL integrada
- Calidad de análisis cruzado
- Recomendaciones accionables

---

**¡Éxito en tu Workshop 2!** 🎓📊📈

---

<details>
<summary>💡 <b>Recursos Adicionales</b></summary>

<br>

**Documentación:**
- [Seaborn Documentation](https://seaborn.pydata.org/)
- [Scipy Stats](https://docs.scipy.org/doc/scipy/reference/stats.html)
- [Sklearn Linear Models](https://scikit-learn.org/stable/modules/linear_model.html)

</details>